In [20]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import os
import time
from typing import List
from pydantic import BaseModel
import openai
from tqdm import tqdm

from dotenv import load_dotenv
load_dotenv()

openai.api_key

In [ ]:
from openai import OpenAI
client = OpenAI()

# completion = client.chat.completions.create(
#     model="gpt-4o",
#     messages=[
#         {"role": "user", "content": "write a haiku about ai"}
#     ]
# );
# print(completion.choices[0].message.content)

Silent circuits hum,  
Wisdom in the code takes form—  
Machines dream of depth.


In [22]:
# 1. want a list of professors.csv
# 2. update professor.csv with the work they do
# 3. for each professor, google their social interests
# 4 update professor.csv with their social interests

In [23]:
def serach(query: str) -> List[str]:
    search_url = "https://html.duckduckgo.com/html/"
    params = {
        'q': query
    }
    headers = {
        'User-Agent': 'Mozilla/5.0'
    }
    try:
        response = requests.post(search_url, data=params, headers=headers, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"Error during DuckDuckGo search for query '{query}': {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for result in soup.find_all('a', {'class': 'result__a'}, href=True):
        links.append(result['href'])
        if len(links) >= 5:
            break
    return links

def extract_text(url: str, max_chars: int = 10000) -> str:
    headers = {
        'User-Agent': 'Mozilla/5.0'
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
        return ""
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    for script in soup(["script", "style"]):
        script.decompose()
        
    text = soup.get_text(separator=" ", strip=True)
    return text[:max_chars]

def get_text(querey: str, max_chars: int = 10000) -> (str, List[str]):
    urls = serach(querey)
    texts = []
    for url in urls:
        text = extract_text(url, max_chars)
        if text:
            texts.append(text)
        time.sleep(1)  # Respectful delay between requests
    combined_text = "\n\n".join(texts)
    return combined_text, urls


In [24]:
class GPTResponseProf(BaseModel):
    name: str
    institution: str
    department: str
    research_areas: List[str]
    
def gpt_request(query: str) -> GPTResponseProf:
    prompt = f"Based on a given university faculty website, fill out the name, institution, department, and the main research areas of the professor: \n\n{query}\n\n Find the following name: \n institution: \n department: \n research_areas: \n\n\n Please provide the output in JSON format ensuring that the types are as follows: name: str     institution: str     department: str     research_areas: List[str] " 
    response = openai.beta.chat.completions.parse(
        model = "gpt-4o-mini", 
        messages=[
            {"role": "system", "content": "You are an assistant that looks at a url and finds information about a professor."},
            {"role": "user", "content": prompt}
        ],
        response_format=GPTResponseProf
    )
    return response.choices[0].message.parsed

In [25]:
print(serach("mit eecs faculty"))


[]
